In [3]:
%pip install imbalanced-learn

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN
from collections import Counter

In [17]:
df_raw = pd.read_csv('/Users/arthurhenrique/Documentos/pessoal/faculdade/documentos/tcc/repository/machine-learning-water-quality/notebook/output/concatenated/wqi_calculado.csv')

In [18]:
df = df_raw[['13_Nitratos_mgl_n)',  '17_OD_mgl_02', '18_PH', '46_Coliformes_Fecais_nmp_100ml', '47_Coliformes_Totais_nmp_100ml', '50_Condutivida_de_Eletrica_us_cm_a_20c', '7_DBO_mgl_02)']]

In [19]:
df = df.dropna()

print(df)

       13_Nitratos_mgl_n)  17_OD_mgl_02  18_PH  \
1                    0.02           7.1   7.64   
2                    0.02           7.1   7.60   
3                    0.14           7.1   6.90   
9                    0.04           6.9   7.20   
10                   0.12           6.9   7.00   
...                   ...           ...    ...   
26510                0.07           7.3   7.00   
26511                0.04           8.5   6.80   
26512                0.05           7.6   6.00   
26513                0.03           7.2   6.00   
26515                0.11           7.5   6.00   

       46_Coliformes_Fecais_nmp_100ml  47_Coliformes_Totais_nmp_100ml  \
1                                50.0                          1700.0   
2                                50.0                          1700.0   
3                               170.0                           300.0   
9                               110.0                          3000.0   
10                              23

In [20]:
df.columns = ['Nitrato', 'OD', 'Ph', 'ColFecais', 'ColTotais', 'Condutividade', 'DBO']

print(df)

       Nitrato   OD    Ph  ColFecais  ColTotais  Condutividade  DBO
1         0.02  7.1  7.64       50.0     1700.0          23.20  2.0
2         0.02  7.1  7.60       50.0     1700.0          23.20  2.0
3         0.14  7.1  6.90      170.0      300.0          16.90  2.0
9         0.04  6.9  7.20      110.0     3000.0          19.60  2.0
10        0.12  6.9  7.00      230.0     1700.0          27.20  2.0
...        ...  ...   ...        ...        ...            ...  ...
26510     0.07  7.3  7.00      800.0        0.0         160.00  2.0
26511     0.04  8.5  6.80      200.0        0.0         106.10  2.0
26512     0.05  7.6  6.00     1300.0        0.0          92.00  2.0
26513     0.03  7.2  6.00       80.0        0.0          70.60  2.0
26515     0.11  7.5  6.00      200.0        0.0          95.08  2.0

[4112 rows x 7 columns]


In [21]:
# Valores ideais (VIdeal)
VIdeal = {'Nitrato': 0, 'OD': 14.6, 'Ph': 7.0, 'ColFecais': 0, 'ColTotais': 0, 'Condutividade': 0, 'DBO': 0}

# Limites permitidos (Si)
Si = {'Nitrato': 45, 'OD': 10, 'Ph': 8.5, 'ColFecais': 100, 'ColTotais': 1000, 'Condutividade': 1000,  'DBO': 5}

# Calcula a constante k
k = 1 / sum(Si.values())

# Função para calcular o qi
def calcular_qi(Vi, VIdeal, Si):
    return 100 * (Vi - VIdeal) / (Si - VIdeal)

# Função para calcular o wi
def calcular_wi(Si, k):
    return k / Si

# Para cada parâmetro, calcula-se o qi e wi
def WQI(df_imputed):
    for param in Si.keys():
        df_imputed[f'qi_{param}'] = calcular_qi(df_imputed[param], VIdeal[param], Si[param])
        df_imputed[f'wi_{param}'] = calcular_wi(Si[param], k)

    df_imputed['WQI'] = df_imputed[[f'qi_{param}' for param in Si.keys()]].multiply(df_imputed[[f'wi_{param}' for param in Si.keys()]].values).sum(axis=1) / df_imputed[[f'wi_{param}' for param in Si.keys()]].sum(axis=1)
    return df_imputed

df = WQI(df)

print(df['WQI'])

1        66.384825
2        65.690542
3        55.898108
9        61.326792
10       60.233739
           ...    
26510    70.571343
26511    48.033421
26512    62.818882
26513    37.737376
26515    38.963891
Name: WQI, Length: 4112, dtype: float64


In [22]:
# Função para classificar o WQI em faixas
def classificar_wqi(wqi):
    if wqi <= 25:
        return 1  # Água Limpa
    elif wqi <= 50:
        return 2  # Não Limpa
    elif wqi <= 75:
        return 3  # Poluída
    else:
        return 4  # Altamente Poluída

# Aplica a função para classificar cada valor de WQI
df['Classificacao'] = df['WQI'].apply(classificar_wqi)

print(df['Classificacao'])

1        3
2        3
3        3
9        3
10       3
        ..
26510    3
26511    2
26512    3
26513    2
26515    2
Name: Classificacao, Length: 4112, dtype: int64


In [23]:
contagem_classificacoes = df['Classificacao'].value_counts().sort_index()

print(contagem_classificacoes)

Classificacao
1      15
2     152
3    1444
4    2501
Name: count, dtype: int64


In [3]:

# dados
df_raw = pd.read_csv('/Users/arthurhenrique/Documentos/pessoal/faculdade/documentos/tcc/repository/machine-learning-water-quality/notebook/output/concatenated/wqi_calculado.csv')

# Separar os dados em variáveis independentes (X) e dependentes (y)
X = df_raw.drop('WQI', axis=1)  # Variáveis independentes
y = df_raw['WQI']  # Variável dependente (rótulo/classe)

# Verificar o balanceamento inicial das classes
print(f"Distribuição original das classes: {Counter(y)}")

# Aplicar ADASYN para balancear as classes
adasyn = ADASYN(random_state=42)
X_res, y_res = adasyn.fit_resample(X, y)

# Verificar o balanceamento após ADASYN
print(f"Distribuição das classes após ADASYN: {Counter(y_res)}")

# Exibir os dados balanceados
df_balanceado = pd.concat([pd.DataFrame(X_res, columns=X.columns), pd.DataFrame(y_res, columns=['Classificacao'])], axis=1)

df_balanceado.to_csv('/Users/arthurhenrique/Documentos/pessoal/faculdade/documentos/tcc/repository/machine-learning-water-quality/notebook/output/amostra_ADASYN.csv', index=False)

Distribuição original das classes: Counter({4: 2501, 3: 1444, 2: 152, 1: 15})
Distribuição das classes após ADASYN: Counter({3: 2682, 4: 2501, 1: 2501, 2: 2493})
